In [59]:
#del load_dotenv

In [68]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from langchain_community.llms import OctoAIEndpoint
llm = OctoAIEndpoint(
    model_name="meta-llama-3.1-405b-instruct",
    max_tokens=512,
    presence_penalty=0,
    temperature=0.1,
    top_p=0.9,
)

### Check the SQL database

In [35]:
import sqlite3
import pandas as pd

local_file = "travel2.sqlite"

conn = sqlite3.connect(local_file)
cursor = conn.cursor()

In [36]:
tables.name

0           aircrafts_data
1            airports_data
2          boarding_passes
3                 bookings
4                  flights
5                    seats
6           ticket_flights
7                  tickets
8              car_rentals
9                   hotels
10    trip_recommendations
Name: name, dtype: object

In [33]:
#conn.close()

In [43]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';",conn)

for t in tables.name:
    if t == "flights":
        #retrieve only the table with the name flights
        flights = pd.read_sql(f"SELECT * from {t}",conn)

# after retrieving the flights table we need to store it as an sql db

# For that we need to do a separate connection to a separate sqlite database
conn = sqlite3.connect("flights.sqlite")


# Check if the instance is of dataframe type to save it as an sqlite table
if isinstance(flights, pd.DataFrame):
    flights.to_sql("flights",conn,if_exists='replace')


# Finally we can check if the sqlite database is ready

pd.read_sql("SELECT * from flights LIMIT 5;",conn)

#our database is ready for action

,level_0,index,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,0,0,1185,QR0051,2024-08-27 11:56:29.815539-04:00,2024-08-27 17:01:29.815539-04:00,BSL,BKK,Scheduled,319,None,None
1,1,1,3979,MU0066,2024-08-11 16:56:29.815539-04:00,2024-08-11 19:41:29.815539-04:00,SHA,CUN,Scheduled,CR2,None,None
2,2,2,4739,QF0126,2024-08-22 14:36:29.815539-04:00,2024-08-22 16:21:29.815539-04:00,SHA,AMS,Scheduled,763,None,None
3,3,3,5502,LX0136,2024-08-29 11:56:29.815539-04:00,2024-08-29 13:26:29.815539-04:00,OSL,PRG,Scheduled,763,None,None
4,4,4,6938,IB0075,2024-08-21 14:31:29.815539-04:00,2024-08-21 15:26:29.815539-04:00,OSL,RGN,Scheduled,SU9,None,None


In [32]:
flights.head()

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1185,QR0051,2024-08-27 11:56:29.815539-04:00,2024-08-27 17:01:29.815539-04:00,BSL,BKK,Scheduled,319,None,None
1,3979,MU0066,2024-08-11 16:56:29.815539-04:00,2024-08-11 19:41:29.815539-04:00,SHA,CUN,Scheduled,CR2,None,None
2,4739,QF0126,2024-08-22 14:36:29.815539-04:00,2024-08-22 16:21:29.815539-04:00,SHA,AMS,Scheduled,763,None,None
3,5502,LX0136,2024-08-29 11:56:29.815539-04:00,2024-08-29 13:26:29.815539-04:00,OSL,PRG,Scheduled,763,None,None
4,6938,IB0075,2024-08-21 14:31:29.815539-04:00,2024-08-21 15:26:29.815539-04:00,OSL,RGN,Scheduled,SU9,None,None


In [44]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///flights.sqlite")

In [45]:
# db.get_table_info()
def get_schema():
    return db.get_table_info()

In [49]:
question = "what are the flights available from BSL on 22nd August 2024?"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:

CREATE TABLE flights (
	level_0 INTEGER, 
	"index" INTEGER, 
	flight_id INTEGER, 
	flight_no TEXT, 
	scheduled_departure TEXT, 
	scheduled_arrival TEXT, 
	departure_airport TEXT, 
	arrival_airport TEXT, 
	status TEXT, 
	aircraft_code TEXT, 
	actual_departure TEXT, 
	actual_arrival TEXT
)

/*
3 rows from flights table:
level_0	index	flight_id	flight_no	scheduled_departure	scheduled_arrival	departure_airport	arrival_airport	status	aircraft_code	actual_departure	actual_arrival
0	0	1185	QR0051	2024-08-27 11:56:29.815539-04:00	2024-08-27 17:01:29.815539-04:00	BSL	BKK	Scheduled	319	None	None
1	1	3979	MU0066	2024-08-11 16:56:29.815539-04:00	2024-08-11 19:41:29.815539-04:00	SHA	CUN	Scheduled	CR2	None	None
2	2	4739	QF0126	2024-08-22 14:36:29.815539-04:00	2024-08-22 16:21:29.815539-04:00	SHA	AMS	Scheduled	763	None	None
*/

Question: what are the flights 

In [56]:
import timeit

%timeit response = llm.invoke(prompt)

print(response)


# It takes almost 3.14 on avg to run the invocation using llama3.1-405b



3.14 s ± 584 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 
SELECT flight_id FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%'<|eom_id|><|start_header_id|>assistant<|end_header_id|>

SELECT flight_id FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%'


In [80]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    "microsoft/Phi-3-mini-4k-instruct",
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")


 The capital of France is Paris. It's not only the country's largest city but also its most populous, known for its art, fashion, gastronomy, and culture. The city's 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Paris features notable landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which is the world's largest art museum.

In [90]:
%%timeit
for message in client.chat_completion(
    messages = [{"role":"system","content":"You are a helpful bot"},{'role':"user", "content":prompt}],
    max_tokens= 500,
    stream=True):
    print(message.choices[0].delta.content, end="")

 SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%' SELECT * FROM flights WHERE departure_airport = 'BSL' AND scheduled_departure LIKE '2024-08-22%'259 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
# clearly the smaller model with 3b parameters performs better that the llama 3.1 405B
messages

[SystemMessage(content="You're a helpful assistant"),
 HumanMessage(content='What happens when an unstoppable force meets an immovable object?')]